<a href="https://colab.research.google.com/github/ManuelaCastano/CuadernosyEjercicios/blob/main/CorrectoRectificadorP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PUNTO 4



In [ ]:
!pip install numpy matplotlib streamlit pyngrok


In [ ]:
%%writefile rectificador.py

#importamos las librerías necesarias
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import streamlit as st
import sympy as sym

# Configurar la página de Streamlit
st.set_page_config(page_title="Rectificador", layout="centered")

st.title("Cálculo del THD y la Distorsión del Factor de Potencia en un Rectificador de Onda Completa")


# Parámetros de simulación ajustables
A = st.number_input('Amplitud de entrada (A)', value=120, step=10)
Fo = st.number_input('Frecuencia de entrada (Fo) en Hz', value=60, step=10)

# Parámetros de simulación
Fs = 30 * Fo  # Frecuencia de muestreo
To = 1 / Fo   # Periodo fundamental
Ts = 1 / Fs   # Periodo de muestreo
t = np.arange(0, 6 * To, Ts)  # Simulando 5 periodos
tau = None
# Selección del circuito: solo resistivo o resistivo-capacitivo
circuito = st.selectbox("Elija el tipo de circuito:", ["Resistivo Puro", "Resistivo-Capacitivo"])

# Parámetros del circuito
if circuito == "Resistivo Puro":
    st.write("""
    En este caso, se analizará el comportamiento de un rectificador de onda completa cuando la carga es solo una resistencia. Esto nos dará una referencia de cómo se comporta el sistema con la configuración más simple.
    """) # st. esto lo muestra en el dashboard
    R_v = st.number_input('Resistencia (R) en ohmios', value=1000, step=100)
    C_v = None  # No hay capacitancia en el circuito resistivo
    tau = R_v  # En el caso resistivo, el tiempo de respuesta depende solo de la resistencia
else:
    st.write("""
    Cuando la carga es un circuito *RC (resistencia y capacitancia en serie)*, el comportamiento cambia debido al almacenamiento de energía en el condensador. El condensador suaviza la señal rectificada, lo que modifica tanto el contenido armónico como el factor de potencia.

    En esta simulación, se permite al usuario variar los valores de *R* y *C* para observar cómo estos afectan tanto el *THD* como la *distorsión del factor de potencia*.
    """)
    R_v = st.number_input('Resistencia (R) en ohmios', value=1000, step=100)
    C_v = st.number_input('Capacitancia (C) en faradios', value=10e-6, format="%.2e",step=10e-6)
    tau = R_v * C_v  # En el circuito RC, el tiempo de respuesta depende de R y C ##

# Función de entrada: señal senoidal rectificada completa
in_o = A * np.sin(2 * np.pi * Fo * t) #Señal sinosoidal de alimentación
in_ = abs(in_o) # Entrada rectificada completa

# Cálculo de la respuesta del circuito
if C_v is None:  # Solo resistivo
    st.write("""
    En una carga resistiva, el contenido armónico es más elevado, ya que la resistencia no tiene la capacidad de filtrar los picos de corriente. El *THD* será generalmente más alto, pero el *factor de potencia* tiende a ser cercano a 1, ya que no hay energía reactiva involucrada.
    """)
    out = in_   # Respuesta del circuito resistivo
else:
  # Función de transferencia para el circuito RC

  #out = sig.lfilter(num, den, in_)  # Salida del circuito RC usando scipy.signal.lfilter
  st.write("""
  Al introducir una capacitancia en la carga, la señal de salida será más suave debido al filtrado que proporciona el condensador. Esto reduce el contenido armónico, disminuyendo el *THD. Sin embargo, la presencia de la capacitancia también introduce energía reactiva, lo que puede reducir el **factor de potencia*.
  """)
  s = sym.symbols('s', complex=True)
  to, R, L, C = sym.symbols('t R L C', positive=True)
  X = sym.Function('X')(s)
  Y = 1/(R*C*s + 1) * X

  to=t

  Y_RC = Y.subs(R, R_v).subs(C, C_v)

  num = np.array([1])  # Numerador
  den = np.array([R_v * C_v, 1])  # Denominador
  G_n = sig.TransferFunction(num, den)

  out = G_n.output(in_, T=to)[1]

# Mostrar gráficos en Streamlit
st.write(f"### Señal de Alimentación ({circuito})")
st.line_chart(in_o)

st.write(f"### Señal Rectificada y Salida del Circuito ({circuito})")
fig, ax = plt.subplots()
plt.plot(t, in_, label='Entrada (In)')
plt.plot(t, out, label='Salida (Out)')
plt.xlabel('Tiempo [s]')
plt.ylabel('Voltaje [V]')
plt.legend()
#plt.show()
st.pyplot(fig)

# Cálculo del espectro de frecuencias (FFT)
vfre = np.fft.rfftfreq(len(out), 1 / Fs)
Xf = np.fft.rfft(out)

st.write("### Espectro de Frecuencias de la Salida (FFT)")
fig, ax = plt.subplots()
ax.stem(vfre, np.abs(Xf) / len(out), use_line_collection=True)
ax.set_xlabel('Frecuencia [Hz]')
ax.set_ylabel('|X(f)|')
st.pyplot(fig)

# Cálculo del THD
def calculate_thd(signal):
    N = len(signal)
    Y = np.fft.rfft(signal)
    Y = 2.0 / N * np.abs(Y[:N // 2])
    fundamental = Y[1]  # Fundamental está en la posición 1 (no 0 debido al DC)
    harmonics = np.sqrt(np.sum(Y[2:]**2))  # Suma de las componentes armónicas
    THD = harmonics / fundamental
    return THD

THD = calculate_thd(out)
st.write(f"### THD (Distorsión Armónica Total) para el circuito {circuito}: {THD:.2%}")

# Cálculo del factor de potencia (FP)
def calculate_power_factor(THD):
  return np.sqrt(1/(1 + THD**2))

FP = calculate_power_factor(THD)
st.write(f"### Factor de Potencia para el circuito {circuito}: {FP:.2f}")

Overwriting rectificador.py


In [ ]:
token = '2lLjOWkqRYefTOvPNoWEGk7ILtC_sTA7Yw6363v3jvkDjZ3P' #colocar aquí su token personal después de crear su cuenta con correo UNAL en Ngrok

In [ ]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run rectificador.py --server.port 9600 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='9600', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://0733-34-106-11-154.ngrok-free.app


In [ ]:
#exit("Stopping the execution")